In [107]:
from pyomo.environ import *

species = {
    'CH6N2':{'C': 1, 'H': 6, 'N': 2},
    'N2O4': {'N': 2, 'O': 4},
    'CO2':  {'C': 1, 'O': 2},
    'NO':   {'N': 1, 'O': 1},
    'H2O':  {'H': 2, 'O': 1},
}



def stoich(species):
    atoms = set()
    for m in species.keys():
        atoms = atoms | molecules[m].keys()

    m = ConcreteModel()

    m.species = Set(initialize=list(species.keys()))
    m.v = Var(m.species, domain=Integers)
    m.y = Var(m.species, domain=Binary)
    m.z = Var()

    m.cons = Constraint(atoms, 
        rule=lambda m, a: sum(m.v[s]*species[s][a] for s in m.species if a in species[s].keys()) == 0)
    m.np = Constraint(m.species, rule=lambda m, s: m.v[s] >= 1-100*(1-m.y[s]))
    m.one = Constraint(expr=sum(m.y[s] for s in m.species) >= 1)
    m.ub = Constraint(m.species, rule=lambda m, s: m.z >= m.v[s])
    m.lb = Constraint(m.species, rule=lambda m, s: m.z >= -m.v[s])
    m.obj = Objective(expr=m.z, sense=minimize)

    SolverFactory('glpk').solve(m)
    m.pprint()
    
stoich(species)

2 Set Declarations
    cons_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        ['C', 'H', 'N', 'O']
    species : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=None
        ['CH6N2', 'CO2', 'H2O', 'N2O4', 'NO']

3 Var Declarations
    v : Size=5, Index=species
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
        CH6N2 :  None :   2.0 :  None : False : False : Integers
          CO2 :  None :  -2.0 :  None : False : False : Integers
          H2O :  None :  -6.0 :  None : False : False : Integers
         N2O4 :  None :   7.0 :  None : False : False : Integers
           NO :  None : -18.0 :  None : False : False : Integers
    y : Size=5, Index=species
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
        CH6N2 :     0 :   0.0 :     1 : False : False : Binary
          CO2 :     0 :   0.0 :     1 : False : False : Binary
          H2O :     0 :   0.0 :     1 : False : False : Binary
         N2O4 :     0 :   1